In [1]:
import sys
import os
import numpy as np
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent))
np.set_printoptions(threshold=sys.maxsize, suppress=True)

In [2]:
from loader import Loader
from constant import FeatureType, FileType, DataType
from copy import deepcopy
from skimage.measure import compare_nrmse
from keras.models import Model
from keras.optimizers import RMSprop
from keras.layers import Input, LSTM, Dense, RepeatVector
import time
import argparse
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import hashlib
import json

### Train, Validate, Test

In [3]:
def encoder_decoder(partition, args):
    X_train, y_train = partition['train']
    X_val, y_val = partition['val']

    batch_size = args.batch_size
    epochs = args.epochs
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=args.early_stop)

    with tf.device('/GPU:0'):        
        encoder_inputs = Input(shape=(args.x_frames * 24, len(args.features)))

        encoder = LSTM(args.hid_dim, return_state=True)
        encoder_outputs, state_h, state_c = encoder(encoder_inputs)
        encoder_states = [state_h, state_c]

        repeat_vector = RepeatVector(24)
        repeat_vector_outputs = repeat_vector(encoder_outputs)

        decoder_lstm = LSTM(args.hid_dim, return_sequences=True, return_state=True)
        decoder_outputs, _, _ = decoder_lstm(repeat_vector_outputs, initial_state=encoder_states)

        decoder_dense = Dense(1, activation=args.activation)
        decoder_outputs = decoder_dense(decoder_outputs)

        rmsprop = RMSprop(lr=args.lr)

        model = Model(encoder_inputs, decoder_outputs)
        model.compile(optimizer=rmsprop, loss=tf.keras.losses.MeanSquaredError(), metrics='accuracy')

        history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
              batch_size=batch_size, epochs=epochs, callbacks=[callback])
        
        print(model.summary())

    return model, history

In [4]:
def test(partition, model, args):
    X_test, y_test = partition['test']
    
    y_pred = model.predict(X_test)
    y_pred = y_pred.reshape(y_test.shape[0], y_test.shape[1])

    nrmse = compare_nrmse(y_test, y_pred)
    return y_pred, nrmse

In [5]:
def experiment(args):
    loader = Loader(args)
    partition, scaler = loader.get_dataset()
    
    ts = time.time()
    model, history = encoder_decoder(partition, args)
    te = time.time()
    
    print('Took {:2.2f} sec for training the model'.format(te-ts))
    
    y_pred, test_acc = test(partition, model, args)
    
    y_test = scaler.inverse_transform(partition['test'][1])
    y_pred = scaler.inverse_transform(y_pred)
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    result['train_losses'] = history.history['loss']
    result['val_losses'] = history.history['val_loss']
    result['train_accs'] = history.history['accuracy']
    result['val_accs'] = history.history['val_accuracy']
    result['train_acc'] = np.mean(history.history['accuracy'])
    result['val_acc'] = np.mean(history.history['val_accuracy'])
    result['test_nrmse'] = test_acc
    result['y_test'] = np.asarray(y_test).tolist()
    result['y_pred'] = np.asarray(y_pred).tolist()
    
#     plt.plot(np.asarray(partition['test'][1]).tolist())
#     plt.plot(np.asarray(y_pred).tolist())
#     plt.legend()
#     plt.show()
    
    setattr(args, 'features', str(args.features))
    return vars(args), model, result

### Manage Experiment

In [6]:
def get_filepath(setting, filetype):
    exp_name = setting['exp_name']
    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
    
    file_format = ""
    directory = ""
    if filetype == FileType.MODEL:
        file_format = 'h5'
        directory = 'models'
    elif filetype == FileType.RESULT:
        file_format = 'json'
        directory = 'results'
        
    filename = "%s-%s.%s" % (exp_name, hash_key, file_format)
    filepath = os.path.join(Path(os.getcwd()).parent.parent, directory, filename)
    return filepath
        
def save_exp_model(setting, model):
    filepath = get_filepath(setting, FileType.MODEL)
    model.save(filepath)

def save_exp_result(setting, result):
    filepath = get_filepath(setting, FileType.RESULT)
    
    result.update(setting)
    with open(filepath, 'w') as f:
        json.dump(result, f)
        
def load_exp_result(exp_name):
    dir_path = os.path.join(Path(os.getcwd()).parent.parent, "results")
    filenames = [f for f in listdeir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result)
    return df

In [7]:
# ====== Random Seed Initialization ====== #
seed = 1234
tf.random.set_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "solar power prediction using weather features"

# ====== Data Loading ====== #
args.plant = 126
args.spot = 174
args.start_date = "20190820"
args.end_date = "20200809"
args.batch_size = 64
args.x_frames = 3
args.y_frames = 1

# ====== Model Capacity ===== #
args.hid_dim = 256

# ====== Optimizer & Training ====== #
args.optim = 'RMSprop'
args.activation = "linear"#'relu'
args.lss = 'MSE'
args.lr = 0.001
args.epochs = 256
args.early_stop = 30
args.evaluation = 'NRMSE'

# ====== Experiment Variable ====== #
name_var1 = 'features'
list_var1 = [FeatureType.SUNSHINE,
                 FeatureType.HUMIDITY,
                 FeatureType.WIND_SPEED,
                 FeatureType.VISIBILITY,
                 FeatureType.GROUND_TEMPERATURE,
                 FeatureType.WIND_DIRECTION,
                 FeatureType.STEAM_PRESSURE,
                 FeatureType.TEMPERATURE,
                 FeatureType.PRECIPITATION,
                 FeatureType.DEW_POINT_TEMPERATURE,
                 FeatureType.ATMOSPHERIC_PRESSURE]

for i in range(len(list_var1)):
    sub_list = list_var1[:i + 1]

    setattr(args, name_var1, str(sub_list))
    print(args)

    setattr(args, name_var1, sub_list)   
    setting, model, result = experiment(deepcopy(args))
    save_exp_model(setting, model)
    save_exp_result(setting, result)
    print(result)
    print()
    
    

Namespace(activation='linear', batch_size=64, early_stop=30, end_date='20200809', epochs=256, evaluation='NRMSE', exp_name='solar power prediction using weather features', features="[<FeatureType.SUNSHINE: '일조(hr)'>]", hid_dim=256, lr=0.001, lss='MSE', optim='RMSprop', plant=126, spot=174, start_date='20190820', x_frames=3, y_frames=1)
train start date: 2019-08-20 00:00:00
train end date: 2020-05-09 00:00:00
val start date: 2020-05-10 00:00:00
val end date: 2020-06-22 00:00:00
test start date: 2020-06-23 00:00:00
test end date: 2020-08-05 00:00:00
train start date: 2019-08-20 00:00:00
train end date: 2020-05-09 00:00:00
val start date: 2020-05-10 00:00:00
val end date: 2020-06-22 00:00:00
test start date: 2020-06-23 00:00:00
test end date: 2020-08-05 00:00:00
Epoch 1/256
5/5 [==============================] - 1s 200ms/step - loss: 0.0798 - accuracy: 0.4940 - val_loss: 0.0172 - val_accuracy: 0.3873
Epoch 2/256
5/5 [==============================] - 0s 21ms/step - loss: 0.0314 - accuracy

Epoch 54/256
5/5 [==============================] - 0s 28ms/step - loss: 0.0143 - accuracy: 0.5387 - val_loss: 0.0315 - val_accuracy: 0.3873
Epoch 55/256
5/5 [==============================] - 0s 20ms/step - loss: 0.0166 - accuracy: 0.5374 - val_loss: 0.0211 - val_accuracy: 0.3873
Epoch 56/256
5/5 [==============================] - 0s 14ms/step - loss: 0.0143 - accuracy: 0.5380 - val_loss: 0.0281 - val_accuracy: 0.3873
Epoch 57/256
5/5 [==============================] - 0s 19ms/step - loss: 0.0163 - accuracy: 0.5388 - val_loss: 0.0177 - val_accuracy: 0.3873
Epoch 58/256
5/5 [==============================] - 0s 28ms/step - loss: 0.0144 - accuracy: 0.5377 - val_loss: 0.0101 - val_accuracy: 0.3873
Epoch 59/256
5/5 [==============================] - 0s 26ms/step - loss: 0.0149 - accuracy: 0.5387 - val_loss: 0.0070 - val_accuracy: 0.3873
Epoch 60/256
5/5 [==============================] - 0s 18ms/step - loss: 0.0146 - accuracy: 0.5379 - val_loss: 0.0094 - val_accuracy: 0.3873
Epoch 61/256


5/5 [==============================] - 0s 28ms/step - loss: 0.0140 - accuracy: 0.5377 - val_loss: 0.0109 - val_accuracy: 0.3873
Epoch 113/256
5/5 [==============================] - 0s 25ms/step - loss: 0.0142 - accuracy: 0.5387 - val_loss: 0.0214 - val_accuracy: 0.3873
Epoch 114/256
5/5 [==============================] - 0s 15ms/step - loss: 0.0137 - accuracy: 0.5368 - val_loss: 0.0400 - val_accuracy: 0.3873
Epoch 115/256
5/5 [==============================] - 0s 27ms/step - loss: 0.0148 - accuracy: 0.5384 - val_loss: 0.0255 - val_accuracy: 0.3873
Epoch 116/256
5/5 [==============================] - 0s 26ms/step - loss: 0.0140 - accuracy: 0.5380 - val_loss: 0.0115 - val_accuracy: 0.3873
Epoch 117/256
5/5 [==============================] - 0s 14ms/step - loss: 0.0140 - accuracy: 0.5374 - val_loss: 0.0208 - val_accuracy: 0.3873
Epoch 118/256
5/5 [==============================] - 0s 13ms/step - loss: 0.0133 - accuracy: 0.5379 - val_loss: 0.0066 - val_accuracy: 0.3873
Epoch 119/256
5/5 [=

Epoch 170/256
5/5 [==============================] - 0s 27ms/step - loss: 0.0129 - accuracy: 0.5374 - val_loss: 0.0108 - val_accuracy: 0.3873
Epoch 171/256
5/5 [==============================] - 0s 29ms/step - loss: 0.0142 - accuracy: 0.5368 - val_loss: 0.0163 - val_accuracy: 0.3873
Epoch 172/256
5/5 [==============================] - 0s 14ms/step - loss: 0.0131 - accuracy: 0.5379 - val_loss: 0.0217 - val_accuracy: 0.3873
Epoch 173/256
5/5 [==============================] - 0s 28ms/step - loss: 0.0136 - accuracy: 0.5379 - val_loss: 0.0069 - val_accuracy: 0.3873
Epoch 174/256
5/5 [==============================] - 0s 28ms/step - loss: 0.0138 - accuracy: 0.5376 - val_loss: 0.0061 - val_accuracy: 0.3873
Epoch 175/256
5/5 [==============================] - 0s 17ms/step - loss: 0.0146 - accuracy: 0.5382 - val_loss: 0.0273 - val_accuracy: 0.3873
Epoch 176/256
5/5 [==============================] - 0s 14ms/step - loss: 0.0133 - accuracy: 0.5372 - val_loss: 0.0113 - val_accuracy: 0.3873
Epoch 

<ipython-input-4-2dfd02194b65>:7: UserWarning: DEPRECATED: skimage.measure.compare_nrmse has been moved to skimage.metrics.normalized_root_mse. It will be removed from skimage.measure in version 0.18.
  nrmse = compare_nrmse(y_test, y_pred)


...

5/5 [==============================] - 0s 18ms/step - loss: 0.0135 - accuracy: 0.5385 - val_loss: 0.0154 - val_accuracy: 0.3873
Epoch 58/256
5/5 [==============================] - 0s 18ms/step - loss: 0.0130 - accuracy: 0.5376 - val_loss: 0.0081 - val_accuracy: 0.3873
Epoch 59/256
5/5 [==============================] - 0s 27ms/step - loss: 0.0124 - accuracy: 0.5388 - val_loss: 0.0097 - val_accuracy: 0.3873
Epoch 60/256
5/5 [==============================] - 0s 24ms/step - loss: 0.0147 - accuracy: 0.5382 - val_loss: 0.0069 - val_accuracy: 0.3873
Epoch 61/256
5/5 [==============================] - 0s 12ms/step - loss: 0.0113 - accuracy: 0.5387 - val_loss: 0.0080 - val_accuracy: 0.3873
Epoch 62/256
5/5 [==============================] - 0s 11ms/step - loss: 0.0152 - accuracy: 0.5377 - val_loss: 0.0068 - val_accuracy: 0.3873
Epoch 63/256
5/5 [==============================] - 0s 11ms/step - loss: 0.0133 - accuracy: 0.5388 - val_loss: 0.0054 - val_accuracy: 0.3873
Epoch 64/256
5/5 [========

5/5 [==============================] - 0s 27ms/step - loss: 0.0121 - accuracy: 0.5388 - val_loss: 0.0123 - val_accuracy: 0.3873
Epoch 116/256
5/5 [==============================] - 0s 25ms/step - loss: 0.0102 - accuracy: 0.5388 - val_loss: 0.0080 - val_accuracy: 0.3873
Epoch 117/256
5/5 [==============================] - 0s 12ms/step - loss: 0.0106 - accuracy: 0.5388 - val_loss: 0.0065 - val_accuracy: 0.3873
Epoch 118/256
5/5 [==============================] - 0s 16ms/step - loss: 0.0093 - accuracy: 0.5384 - val_loss: 0.0121 - val_accuracy: 0.3873
Epoch 119/256
5/5 [==============================] - 0s 27ms/step - loss: 0.0092 - accuracy: 0.5388 - val_loss: 0.0191 - val_accuracy: 0.3873
Epoch 120/256
5/5 [==============================] - 0s 25ms/step - loss: 0.0126 - accuracy: 0.5384 - val_loss: 0.0071 - val_accuracy: 0.3873
Epoch 121/256
5/5 [==============================] - 0s 13ms/step - loss: 0.0110 - accuracy: 0.5388 - val_loss: 0.0110 - val_accuracy: 0.3873
Epoch 122/256
5/5 [=

Epoch 173/256
5/5 [==============================] - 0s 12ms/step - loss: 0.0078 - accuracy: 0.5388 - val_loss: 0.0093 - val_accuracy: 0.3873
Epoch 174/256
5/5 [==============================] - 0s 15ms/step - loss: 0.0074 - accuracy: 0.5388 - val_loss: 0.0157 - val_accuracy: 0.3873
Epoch 175/256
5/5 [==============================] - 0s 27ms/step - loss: 0.0090 - accuracy: 0.5388 - val_loss: 0.0134 - val_accuracy: 0.3873
Epoch 176/256
5/5 [==============================] - 0s 26ms/step - loss: 0.0091 - accuracy: 0.5388 - val_loss: 0.0086 - val_accuracy: 0.3873
Epoch 177/256
5/5 [==============================] - 0s 12ms/step - loss: 0.0084 - accuracy: 0.5388 - val_loss: 0.0140 - val_accuracy: 0.3873
Epoch 178/256
5/5 [==============================] - 0s 27ms/step - loss: 0.0073 - accuracy: 0.5388 - val_loss: 0.0166 - val_accuracy: 0.3873
Epoch 179/256
5/5 [==============================] - 0s 27ms/step - loss: 0.0080 - accuracy: 0.5388 - val_loss: 0.0089 - val_accuracy: 0.3873
Epoch 

Epoch 231/256
5/5 [==============================] - 0s 11ms/step - loss: 0.0054 - accuracy: 0.5388 - val_loss: 0.0093 - val_accuracy: 0.3873
Epoch 232/256
5/5 [==============================] - 0s 11ms/step - loss: 0.0059 - accuracy: 0.5388 - val_loss: 0.0080 - val_accuracy: 0.3873
Epoch 233/256
5/5 [==============================] - 0s 11ms/step - loss: 0.0060 - accuracy: 0.5388 - val_loss: 0.0174 - val_accuracy: 0.3873
Epoch 234/256
5/5 [==============================] - 0s 11ms/step - loss: 0.0069 - accuracy: 0.5384 - val_loss: 0.0095 - val_accuracy: 0.3873
Epoch 235/256
5/5 [==============================] - 0s 11ms/step - loss: 0.0047 - accuracy: 0.5388 - val_loss: 0.0108 - val_accuracy: 0.3873
Epoch 236/256
5/5 [==============================] - 0s 10ms/step - loss: 0.0042 - accuracy: 0.5388 - val_loss: 0.0097 - val_accuracy: 0.3873
Epoch 237/256
5/5 [==============================] - 0s 12ms/step - loss: 0.0036 - accuracy: 0.5388 - val_loss: 0.0153 - val_accuracy: 0.3873
Epoch 

<ipython-input-4-2dfd02194b65>:7: UserWarning: DEPRECATED: skimage.measure.compare_nrmse has been moved to skimage.metrics.normalized_root_mse. It will be removed from skimage.measure in version 0.18.
  nrmse = compare_nrmse(y_test, y_pred)
